In [9]:
import numpy as np
import pandas as pd
import os
import shutil
import itertools

# Read 

In [16]:
colpo_meta = pd.read_excel(r"C:\Users\taral\OneDrive\Documents\Major project\Data\IARC\Colposcopic images\Colposcopic Cases Meta data.xlsx" )

colpo_meta.columns = colpo_meta.iloc[0]

colpo_meta = colpo_meta.iloc[1:].reset_index(drop=True)
colpo_meta = colpo_meta[["Case Number", "CaseID", "\nProvisional diagnosis"]]

EXCLUDE  = [i for i in colpo_meta["\nProvisional diagnosis"].unique() if any(w in i.lower() for w in ["Inadequate colposcopy", "inadequate colposcopy"])]
print(EXCLUDE)
colpo_meta = colpo_meta[~colpo_meta["\nProvisional diagnosis"].isin(EXCLUDE)]
print(colpo_meta["\nProvisional diagnosis"].unique)

NORMAL_CLASSES = [
    "Type 1 transformation zone; normal",
"Type 2 transformation zone; normal",
"Type 3 transformation zone; normal",
"Type 2 transformation zone; normal cervix with atrophic change",
"Type 3 transformation zone; normal cervix with atrophic change",
"Type 1 transformation zone; normal with pregnancy-induced changes",
"Type 1 transformation zone; normal with ectropion",
"Type1 transformation zone; normal with ectropion",
"Type 1 transformation zone; normal with evidence of candida infection",
"Type 1 transformation zone; normal with evidence of trichomoniasis",
"Type 1 transformation zone; normal with evidence of <i>Trichomonas vaginalis</i>",
"Type 1 transformation zone; normal with endocervical polyp",
"Tuberculosis ulcer of cervix healed",
"Type 2 transformation zone; SPI",
"Type 1 transformation zone; subclinical papilloma virus infection (SPI)",
"Type 1 transformation zone; subclinical papillomavirus infection (SPI)",
"Type 1 transformation zone; congenital transformation zone"
]

ABNORMAL_CLASSES = [i for i in colpo_meta["\nProvisional diagnosis"].unique() if i not in NORMAL_CLASSES]
print("\n".join(ABNORMAL_CLASSES))

['Inadequate colposcopy due to severe vaginal infection', 'Inadequate colposcopy']
<bound method Series.unique of 0                     Type 1 transformation zone; normal
1                     Type 1 transformation zone; normal
2                     Type 1 transformation zone; normal
3                     Type 1 transformation zone; normal
4                     Type 2 transformation zone; normal
                             ...                        
194    Type 2 transformation zone; high-grade squamou...
195    Recurrence of high-grade squamous intraepithel...
196                               Recurrence after LLETZ
197                   Type 1 transformation zone; normal
198                   Type 1 transformation zone; normal
Name: \nProvisional diagnosis, Length: 198, dtype: object>
Type 1 transformation zone; low-grade Squamous intraepithelial lesion (LSIL) with ectropion
Type 1 transformation zone; low-grade squamous intraepithelial lesion (LSIL)
Type 1 transformation zone; cer

In [17]:
colpo_image_dat = pd.read_excel(r"C:\Users\taral\OneDrive\Documents\Major project\Data\IARC\Colposcopic images\Cases - Images.xlsx")
colpo_image_dat= colpo_image_dat[["Case Number", "File", "Type"]]
colpo_image_dat.head(2)

,Case Number,File,Type
0,1,AAAW1.jpg,After normal saline
1,1,AAAW3.jpg,After acetic acid


In [18]:
def find_matches(row, df2):
    matches = df2[df2['File'].str.startswith(row['CaseID'])]
    return matches

In [19]:
results = []

for _, row in colpo_meta.iterrows():
    matches = find_matches(row, colpo_image_dat)
    for _, match in matches.iterrows():
        print("FALSE:" + str(row['Case Number'] ) if row['Case Number'] != match['Case Number'] else "", end = "")
        results.append({
            'File': match['File'],
            'CaseID': row['CaseID'],
            'Case Number': row['Case Number'],
            'Type': match['Type'],
            'Class': row['\nProvisional diagnosis']
        })

final_colpo_set = pd.DataFrame(results)

In [20]:
final_colpo_set.head()

,File,CaseID,Case Number,Type,Class
0,AAAW1.jpg,AAAW,1,After normal saline,Type 1 transformation zone; normal
1,AAAW3.jpg,AAAW,1,After acetic acid,Type 1 transformation zone; normal
2,AAAW2.jpg,AAAW,1,After acetic acid with green filter,Type 1 transformation zone; normal
3,AAAW4.jpg,AAAW,1,After Lugol’s iodine,Type 1 transformation zone; normal
4,AADE0.jpg,AADE,2,After acetic acid,Type 1 transformation zone; normal


In [21]:
final_colpo_set['Class'].value_counts()

Class
Type 1 transformation zone; normal                                                                                      220
Type 2 transformation zone; high-grade squamous intraepithelial lesion (HSIL)                                            83
Type 3 transformation zone; high-grade squamous intraepithelial lesion (HSIL)                                            80
Type 1 transformation zone; high-grade squamous intraepithelial lesion (HSIL)                                            53
Type 1 transformation zone; low-grade squamous intraepithelial lesion (LSIL)                                             33
Type 1 transformation zone; congenital transformation zone                                                               31
Type 1 transformation zone; normal with ectropion                                                                        21
Type 3 transformation zone; suspicion of invasive squamous cell carcinoma                                                18
Ty

# Move data

In [22]:
def copy_images(from_to):
    c = 0
    for fr, to in from_to:
        shutil.copy(fr, to)
        c +=1
        print(c, "done")

In [23]:
BASE_PATH_FROM = r"C:\Users\taral\OneDrive\Documents\Major project\Data\IARC\Colposcopic images"
BASE_PATH_TO = r"C:\Users\taral\OneDrive\Documents\Major project\Data\DEEPTEST"

In [24]:
from_to = []
for _,row in final_colpo_set.iterrows():
    from_to.append((BASE_PATH_FROM + "\\Case {:03}\\{}".format(
            row["Case Number"],
            row["File"]
            ),
          BASE_PATH_TO + "\\" + row['File']))
from_to[:2]

[('C:\\Users\\taral\\OneDrive\\Documents\\Major project\\Data\\IARC\\Colposcopic images\\Case 001\\AAAW1.jpg',
  'C:\\Users\\taral\\OneDrive\\Documents\\Major project\\Data\\DEEPTEST\\AAAW1.jpg'),
 ('C:\\Users\\taral\\OneDrive\\Documents\\Major project\\Data\\IARC\\Colposcopic images\\Case 001\\AAAW3.jpg',
  'C:\\Users\\taral\\OneDrive\\Documents\\Major project\\Data\\DEEPTEST\\AAAW3.jpg')]

In [25]:
# generate image list for Normal folder
normal_path_to = BASE_PATH_TO + "\\Normal\\"
normal_from_to = []
for class_ in NORMAL_CLASSES:
    rows = final_colpo_set[final_colpo_set['Class'] == class_]
    paths = [
        ( BASE_PATH_FROM + "\\Case {:03}\\{}".format(
            row["Case Number"],
            row["File"]
            ),
          normal_path_to + row['File'])
            for _, row in rows.iterrows()]
    normal_from_to.extend(paths)
normal_from_to[:2]

[('C:\\Users\\taral\\OneDrive\\Documents\\Major project\\Data\\IARC\\Colposcopic images\\Case 001\\AAAW1.jpg',
  'C:\\Users\\taral\\OneDrive\\Documents\\Major project\\Data\\DEEPTEST\\Normal\\AAAW1.jpg'),
 ('C:\\Users\\taral\\OneDrive\\Documents\\Major project\\Data\\IARC\\Colposcopic images\\Case 001\\AAAW3.jpg',
  'C:\\Users\\taral\\OneDrive\\Documents\\Major project\\Data\\DEEPTEST\\Normal\\AAAW3.jpg')]

In [26]:
copy_images(normal_from_to)

1 done
2 done
3 done
4 done
5 done
6 done
7 done
8 done
9 done
10 done
11 done
12 done
13 done
14 done
15 done
16 done
17 done
18 done
19 done
20 done
21 done
22 done
23 done
24 done
25 done
26 done
27 done
28 done
29 done
30 done
31 done
32 done
33 done
34 done
35 done
36 done
37 done
38 done
39 done
40 done
41 done
42 done
43 done
44 done
45 done
46 done
47 done
48 done
49 done
50 done
51 done
52 done
53 done
54 done
55 done
56 done
57 done
58 done
59 done
60 done
61 done
62 done
63 done
64 done
65 done
66 done
67 done
68 done
69 done
70 done
71 done
72 done
73 done
74 done
75 done
76 done
77 done
78 done
79 done
80 done
81 done
82 done
83 done
84 done
85 done
86 done
87 done
88 done
89 done
90 done
91 done
92 done
93 done
94 done
95 done
96 done
97 done
98 done
99 done
100 done
101 done
102 done
103 done
104 done
105 done
106 done
107 done
108 done
109 done
110 done
111 done
112 done
113 done
114 done
115 done
116 done
117 done
118 done
119 done
120 done
121 done
122 done
123 done
1

In [29]:
# generate image list for Normal folder
abnormal_path_to = BASE_PATH_TO + "\\Abnormal\\"
abnormal_from_to = []
for class_ in ABNORMAL_CLASSES:
    rows = final_colpo_set[final_colpo_set['Class'] == class_]
    paths = [
        ( BASE_PATH_FROM + "\\Case {:03}\\{}".format(
            row["Case Number"],
            row["File"]
            ),
          abnormal_path_to + row['File'])
            for _, row in rows.iterrows()]
    abnormal_from_to.extend(paths)
abnormal_from_to[:2]

[('C:\\Users\\taral\\OneDrive\\Documents\\Major project\\Data\\IARC\\Colposcopic images\\Case 065\\AAFG1.jpg',
  'C:\\Users\\taral\\OneDrive\\Documents\\Major project\\Data\\DEEPTEST\\Abnormal\\AAFG1.jpg'),
 ('C:\\Users\\taral\\OneDrive\\Documents\\Major project\\Data\\IARC\\Colposcopic images\\Case 065\\AAFG0.jpg',
  'C:\\Users\\taral\\OneDrive\\Documents\\Major project\\Data\\DEEPTEST\\Abnormal\\AAFG0.jpg')]

In [30]:
copy_images(abnormal_from_to)

1 done
2 done
3 done
4 done
5 done
6 done
7 done
8 done
9 done
10 done
11 done
12 done
13 done
14 done
15 done
16 done
17 done
18 done
19 done
20 done
21 done
22 done
23 done
24 done
25 done
26 done
27 done
28 done
29 done
30 done
31 done
32 done
33 done
34 done
35 done
36 done
37 done
38 done
39 done
40 done
41 done
42 done
43 done
44 done
45 done
46 done
47 done
48 done
49 done
50 done
51 done
52 done
53 done
54 done
55 done
56 done
57 done
58 done
59 done
60 done
61 done
62 done
63 done
64 done
65 done
66 done
67 done
68 done
69 done
70 done
71 done
72 done
73 done
74 done
75 done
76 done
77 done
78 done
79 done
80 done
81 done
82 done
83 done
84 done
85 done
86 done
87 done
88 done
89 done
90 done
91 done
92 done
93 done
94 done
95 done
96 done
97 done
98 done
99 done
100 done
101 done
102 done
103 done
104 done
105 done
106 done
107 done
108 done
109 done
110 done
111 done
112 done
113 done
114 done
115 done
116 done
117 done
118 done
119 done
120 done
121 done
122 done
123 done
1